# Mini Problem Set: Diagnosis
## Implicats and Implicants for Constraint Programming (100 points)

1. [Introduction](#introduction)
2. [Prime implicate method](#prime-implicate)
    1. [Detect conflicts](#conflict)
    2. [Test validity](#validity)
    3. [Test unsatisfiability](#unsatisfiable)
    4. [Full algorithm](#full-algorithm)

Make sure you load the dependencies below by highlighting the cell below and pressing Shift + Enter.

In [20]:
%load_ext autoreload
%autoreload 2
from propositional_state_logic import *
from sat_solver import *
from utils import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Introduction <a id="introduction"/>
</a>

In this problem set, you'll implement a prime implicate generator. This generator helps decrease the size of a model to make diagnosis more efficient.

## Prime implicate method <a id="prime-implicate"/></a>

We describe the thruster model based on the "Diagnosis: Implicates and Implicants for Constraint Programming", which can be found in the tutorial. We've used tools from the Pset 3: Diagnosis, and included some additional ones to facilitate the implementation. First, we will define the model and constraints. Next, we will create stand alone methods to identify if a node is valid, if it is unsatisfiable, or if it is satisfiable. Based on these methods, we will write the full algorithm to generate prime implicates, based on the minimal conflicts that we find. 

In [21]:
p = Problem()

# Define the variables for the mini thruster problem with variables: T1, R1, and P3. Returns a Variable object.
# Thrust: T1
T1 = p.add_variable('thruster', type='finite_domain', domain=['thrust', 'nothrust'])
# Thruster: R1
R1 = p.add_variable('runthruster', type='finite_domain', domain=['on', 'off'])
# Pressure before the thruster: P3
P3 = p.add_variable('pressure', type='finite_domain', domain=['high', 'low'])

# Add the theory / problem constraints.
# The thruster only outputs thrust when it is on and when the input from P3 is high
p.add_constraint('runthruster=on & pressure=high => thruster=thrust')
p.add_constraint('runthruster=on & pressure=low => thruster=nothrust')
p.add_constraint('runthruster=off => thruster=nothrust')

# Prints out constraints nicely in LaTeX, so you can check them.
display_constraints(p)

# Define SAT for future use
sat = SATSolver(p)

Constraints:


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In the following block of code, we create the structure of the tree over which we will iterate. In a true sceneario, we would not be searching over all of the tree (to save time), but for the purpose of this exercise, we iterate over all the tree to present the different statuses that a node can have: valid, unsatifiable, satisfiable.

In [22]:
thruster_model = {
    frozenset([T1.get_assignment('thrust')]) : {
        frozenset([T1.get_assignment('thrust'), R1.get_assignment('on')]) : {
            frozenset([T1.get_assignment('thrust'), R1.get_assignment('on'), P3.get_assignment('high')]) : {},
            frozenset([T1.get_assignment('thrust'), R1.get_assignment('on'), P3.get_assignment('low')]) : {},
            },
        frozenset([T1.get_assignment('thrust'), R1.get_assignment('off')]) : {
            frozenset([T1.get_assignment('thrust'), R1.get_assignment('off'), P3.get_assignment('high')]) : {},
            frozenset([T1.get_assignment('thrust'), R1.get_assignment('off'), P3.get_assignment('low')]) : {},
            },
        frozenset([T1.get_assignment('thrust'), P3.get_assignment('high')]) : {},
        frozenset([T1.get_assignment('thrust'), P3.get_assignment('low')]) : {},
        },
    
    frozenset([T1.get_assignment('nothrust')]) : {
        frozenset([T1.get_assignment('nothrust'), R1.get_assignment('on')]) : {
            frozenset([T1.get_assignment('nothrust'), R1.get_assignment('on'), P3.get_assignment('high')]) : {},
            frozenset([T1.get_assignment('nothrust'), R1.get_assignment('on'), P3.get_assignment('low')]) : {},
            },
        frozenset([T1.get_assignment('nothrust'), R1.get_assignment('off')]) : {
            frozenset([T1.get_assignment('nothrust'), R1.get_assignment('off'), P3.get_assignment('high')]) : {},
            frozenset([T1.get_assignment('nothrust'), R1.get_assignment('off'), P3.get_assignment('low')]) : {},
            },
        frozenset([T1.get_assignment('nothrust'), P3.get_assignment('high')]) : {},
        frozenset([T1.get_assignment('nothrust'), P3.get_assignment('low')]) : {},
        },
    
    frozenset([R1.get_assignment('on')]) : {
            frozenset([R1.get_assignment('on'), P3.get_assignment('high')]) : {},
            frozenset([R1.get_assignment('on'), P3.get_assignment('low')]) : {},
            },
        frozenset([R1.get_assignment('off')]) : {
            frozenset([R1.get_assignment('off'), P3.get_assignment('high')]) : {},
            frozenset([R1.get_assignment('off'), P3.get_assignment('low')]) : {},
            },
    
    frozenset([P3.get_assignment('high')]) : {},
    
    frozenset([P3.get_assignment('low')]) : {},
    }

### Detect conflicts <a id="conflict"/>
</a>



One of the pruning rules that we have in our tree is to check for conflicts. Thanks to the helper function $check_consistency$ that was provided to us for pset 3, it is easy to program:

In [23]:
def is_conflict(sat, candidate):
    return not sat.check_consistency(candidate)[0]

We check our function using assignments for which we know the result: 

In [24]:
# This should be true
is_conflict(sat, frozenset([T1.get_assignment('thrust'), R1.get_assignment('off')]))

True

In [25]:
# This should be false
is_conflict(sat, frozenset([T1.get_assignment('thrust'), R1.get_assignment('on'), P3.get_assignment('high')]))

False

### Test validity <a id="validity"/>
</a>

For the second pruning rule, we need to identify if a candidate is valid. In order to do so, we need to check if it is not a conflict, and if all of it children also aren't conflicts. To do so, we will use a helper function to get all of the children (keys) of a node, based on the structure of the tree we created above.

In [26]:
# Helper Function
def get_all_keys(d):
    for key, value in d.items():
        yield key
        if isinstance(value, dict):
            yield from get_all_keys(value)


Now that we have this gelper function, we can check for validity for a candidate:

In [27]:
# Problem 1: Test Validity
def is_valid(candidate, sat, thruster_model_children):
    # list of all the following partial assignments
    childCandidateList = list(get_all_keys(thruster_model_children))
    
    for child in childCandidateList:
        if is_conflict(sat, child):
            return False
    return True


We run a series of test on candidates that we know are valid or not:

In [28]:
# Validity Test 1: Candidate that is Not Valid
candidate = frozenset([T1.get_assignment('nothrust')])
thruster_model_subset_1 = {
        frozenset([T1.get_assignment('nothrust'), R1.get_assignment('on')]) : {
            frozenset([T1.get_assignment('nothrust'), R1.get_assignment('on'), P3.get_assignment('high')]) : {},
            frozenset([T1.get_assignment('nothrust'), R1.get_assignment('on'), P3.get_assignment('low')]) : {},
            },
        frozenset([T1.get_assignment('nothrust'), R1.get_assignment('off')]) : {
            frozenset([T1.get_assignment('nothrust'), R1.get_assignment('off'), P3.get_assignment('high')]) : {},
            frozenset([T1.get_assignment('nothrust'), R1.get_assignment('off'), P3.get_assignment('low')]) : {},
            },
        frozenset([T1.get_assignment('nothrust'), P3.get_assignment('high')]) : {},
        frozenset([T1.get_assignment('nothrust'), P3.get_assignment('low')]) : {},
        }

is_valid(candidate, sat, thruster_model_subset_1)

False

In [29]:
# Validity Test 2: Candidate that is Valid
candidate = frozenset([T1.get_assignment('nothrust'), R1.get_assignment('off')])
thruster_model_subset = thruster_model_subset_1[candidate]

is_valid(candidate, sat, thruster_model_subset)

True

### Test Unsatisfiability <a id="unsatisfiable"/>
</a>

We now test for unsatisfiability. Similar to the test for validity, we need to check if a candidate is a conflict, and if all of it children are also conflicts.

In [30]:
# Problem 2: Test for Unsatisfiability
# NOTE: IF THE CANDIDATE ITSELF CONFLICTS WITH THE THEORY, THE CANDIDATE IS UNSATISFIABLE
def is_unsatisfiable(candidate, sat, thruster_model_children):
    # list of all the following partial assignments
    childCandidateList = list(get_all_keys(thruster_model_children))
    
    if is_conflict(sat, candidate):
        return True
    
    for child in childCandidateList:
        if not is_conflict(sat, child): # if one of the children is consistent
            return False
    return True

We also run a series of test on candidates that we know are unsatisfiable or not:

In [31]:
# Unsatisfiability Test 1: Candidate that is Unsatisfiable
candidate = frozenset([T1.get_assignment('thrust'), R1.get_assignment('off')])
thruster_model_subset_3 = {
            frozenset([T1.get_assignment('thrust'), R1.get_assignment('off'), P3.get_assignment('high')]) : {},
            frozenset([T1.get_assignment('thrust'), R1.get_assignment('off'), P3.get_assignment('low')]) : {},
            }

is_unsatisfiable(candidate, sat, thruster_model_subset_3)

True

In [32]:
# Unsatisfiability Test 2: Candidate that is Not Unsatisfiable
candidate = frozenset([T1.get_assignment('thrust')])
thruster_model_subset_4 = {
        frozenset([T1.get_assignment('thrust'), R1.get_assignment('on')]) : {
            frozenset([T1.get_assignment('thrust'), R1.get_assignment('on'), P3.get_assignment('high')]) : {},
            frozenset([T1.get_assignment('thrust'), R1.get_assignment('on'), P3.get_assignment('low')]) : {},
            },
        frozenset([T1.get_assignment('thrust'), R1.get_assignment('off')]) : {
            frozenset([T1.get_assignment('thrust'), R1.get_assignment('off'), P3.get_assignment('high')]) : {},
            frozenset([T1.get_assignment('thrust'), R1.get_assignment('off'), P3.get_assignment('low')]) : {},
            },
        frozenset([T1.get_assignment('thrust'), P3.get_assignment('high')]) : {},
        frozenset([T1.get_assignment('thrust'), P3.get_assignment('low')]) : {},
        }

is_unsatisfiable(candidate, sat, thruster_model_subset_4)

False

### Full algorithm: Minimal conflict generator <a id="full-algorithm"/>
</a>

In [33]:
def prime_implicate_finder(sat, candidateList):
    ## sat represents the theory behind the represented model
    ## CandidateList: List of Tuples (candidate, thruster_model_children), 
        # where the candidate is the item to be tested and 
        # where thruster_model_children is a dictionary holding the children of the given candidate
    ## generatorAdditions: add candidate to generatorAdditions list when it should go to the generator
    ## convertToImplicate: add candidate to the convertToImplicateList when it needs to be converted to an implicate before being passed to the generator
    ## solutions: add candidate to the solutions list when a candidate is a minimal conflict
    ### Function should return a tuple in the format (generatorAdditions, convertToImplicate, solutions)
    
    generatorAdditions = []
    convertToImplicate = []
    solutions = []
    
    ### YOUR CODE HERE ###
    
    for candidateTuple in candidateList: 
        candidate = candidateTuple[0]
        thruster_model_children = candidateTuple[1]
        
        if is_valid(candidate, sat, thruster_model_children):
            generatorAdditions.append(candidate)
        elif is_unsatisfiable(candidate, sat, thruster_model_children):
            solutions.append(candidate)
        else: #candidate is satisfiable
            convertToImplicate.append(candidate)
            
    return (generatorAdditions, convertToImplicate, solutions)
            

In [34]:
#Example Problem

candidate1 = frozenset([T1.get_assignment('thrust')])
candidate2 = frozenset([T1.get_assignment('nothrust')])
candidate4 = frozenset([R1.get_assignment('off')])
candidate1A = frozenset([T1.get_assignment('thrust'), R1.get_assignment('on')])
candidate1B = frozenset([T1.get_assignment('thrust'), R1.get_assignment('off')])
candidate2A = frozenset([T1.get_assignment('nothrust'), R1.get_assignment('on')])
candidate2B = frozenset([T1.get_assignment('nothrust'), R1.get_assignment('off')])

candidateList = [
(candidate1, thruster_model[candidate1]),
(candidate2, thruster_model[candidate2]),
(candidate4, thruster_model[candidate4]),
(candidate1A,thruster_model[candidate1][candidate1A]),
(candidate1B,thruster_model[candidate1][candidate1B]),
(candidate2A,thruster_model[candidate2][candidate2A]),
(candidate2B,thruster_model[candidate2][candidate2B]),
]

In [35]:
#Test Prime Implicate Algorithm Output: 

(generatorAdditions, convertToImplicate, solutions) = prime_implicate_finder(sat, candidateList)
print("Generator Additions: ", generatorAdditions, "\n")
print("Convert To Implicate: ", convertToImplicate, "\n")
print("Solutions: ", solutions, "\n")

Generator Additions:  [frozenset({(runthruster = off)}), frozenset({(runthruster = off), (thruster = nothrust)})] 

Convert To Implicate:  [frozenset({(thruster = thrust)}), frozenset({(thruster = nothrust)}), frozenset({(runthruster = on), (thruster = thrust)}), frozenset({(thruster = nothrust), (runthruster = on)})] 

Solutions:  [frozenset({(runthruster = off), (thruster = thrust)})] 

